In [3]:
import numpy as np
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json
from datetime import datetime

In [4]:
# Load the API key from the .env file
load_dotenv()
api_key = os.getenv("API_KEY")

# Fetch the odds data
sport_key = "basketball_ncaab"
regions = "eu"  # Regions: us, us2, uk, eu, au
markets = "h2h"  # Betting markets
odds_format = "decimal"  # Odds formats: decimal, american

url = f"https://api.the-odds-api.com/v4/sports/{sport_key}/odds"
params = {
    "apiKey": api_key,
    "regions": regions,
    "markets": markets,
    "oddsFormat": odds_format
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    
    # Add timestamp and parameters to the data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for game in data:
        game["timestamp"] = timestamp
    
    # Generate a unique filename using a timestamp
    filename = f"odds_data_{datetime.now().strftime('%Y%m%d%H%M%S')}_{sport_key}_{regions}_{markets}_{odds_format}.json"
    
    # Save the JSON response to a file
    with open("json_requests/"+filename, "w") as json_file:
        json.dump(data, json_file, indent=4)
    
else:
    print(f"Error: {response.status_code} - {response.text}")

In [ ]:
# Prepare list for flattened data entries
flattened_data = []
for game in data:
    base_info = {
        "id": game["id"],
        "sport_key": game["sport_key"],
        "sport_title": game["sport_title"],
        "commence_time": game["commence_time"],
        "home_team": game["home_team"],
        "away_team": game["away_team"],
        "timestamp": game["timestamp"]
    }
    
    # Process each bookmaker and its markets
    for bookmaker in game["bookmakers"]:
        bookmaker_key = bookmaker["key"]
        for market in bookmaker["markets"]:
            market_key = market["key"]
            
            entry = base_info.copy()
            for outcome in market["outcomes"]:
                team = "home" if outcome["name"] == game["home_team"] else "away"
                # Generalized column name based on market type and team
                column_name = f"{bookmaker_key}_{team}_{market_key}_odds"
                entry[column_name] = outcome["price"]
            
            # Add the entry to flattened_data
            flattened_data.append(entry)

# Convert the flattened data into a DataFrame
df = pd.DataFrame(flattened_data).drop_duplicates()

# Collapse rows for each fixture by using `groupby` and `agg`
df = df.groupby(
    ["id", "sport_key", "sport_title", "commence_time", "home_team", "away_team", "timestamp"],
    as_index=False
).agg("first")  # Use "first" to take the first non-null value in each group

,id,sport_key,sport_title,commence_time,home_team,away_team,timestamp,betonlineag_away_h2h_odds,betonlineag_home_h2h_odds,pinnacle_away_h2h_odds,...,marathonbet_away_h2h_odds,marathonbet_home_h2h_odds,livescorebet_eu_away_h2h_odds,livescorebet_eu_home_h2h_odds,everygame_away_h2h_odds,everygame_home_h2h_odds,betfair_ex_eu_home_h2h_odds,betfair_ex_eu_away_h2h_odds,betfair_ex_eu_home_h2h_lay_odds,betfair_ex_eu_away_h2h_lay_odds
0,033ad593451747fad8839dba527324c7,basketball_ncaab,NCAAB,2024-11-24T18:30:09Z,Duquesne Dukes,South Dakota St Jackrabbits,2024-11-24 15:02:01,1.72,2.18,1.68,...,1.69,2.13,1.71,2.16,1.77,2.10,NaN,NaN,NaN,NaN
1,0445691004e7c57f87754cb1489bec44,basketball_ncaab,NCAAB,2024-11-24T18:00:00Z,Wright St Raiders,South Florida Bulls,2024-11-24 15:02:01,1.65,2.35,1.61,...,1.68,2.15,1.61,2.35,1.65,2.35,NaN,NaN,NaN,NaN
2,0667d6a6d373bb3e590fb41227e2e48b,basketball_ncaab,NCAAB,2024-11-24T22:00:00Z,Lipscomb Bisons,Jackson St Tigers,2024-11-24 15:02:01,NaN,NaN,NaN,...,11.50,1.03,9.00,1.07,NaN,NaN,NaN,NaN,NaN,NaN
3,0b083afb99e8388bdc93ae163f1cbc35,basketball_ncaab,NCAAB,2024-11-25T00:30:20Z,Boston College Eagles,Old Dominion Monarchs,2024-11-24 15:02:01,6.25,1.15,5.90,...,5.05,1.16,4.75,1.19,5.40,1.17,NaN,NaN,NaN,NaN
4,1635b74ee3d85db1ce347e6a46d1774e,basketball_ncaab,NCAAB,2024-11-25T01:30:00Z,Vanderbilt Commodores,Drake Bulldogs,2024-11-24 15:02:01,2.55,1.57,2.46,...,2.51,1.51,2.43,1.57,2.60,1.56,NaN,NaN,NaN,NaN
5,209e7bf2e35c4b23cb9267563b8f6b91,basketball_ncaab,NCAAB,2024-11-24T15:30:00Z,Princeton Tigers,Portland Pilots,2024-11-24 15:02:01,6.20,1.14,6.23,...,5.00,1.16,4.50,1.21,5.70,1.15,NaN,NaN,NaN,NaN
6,2f84bd99388561173bea4f31f57ebc07,basketball_ncaab,NCAAB,2024-11-24T19:00:00Z,Valparaiso Beacons,Eastern Illinois Panthers,2024-11-24 15:02:01,3.30,1.36,3.33,...,3.36,1.31,3.10,1.38,3.35,1.36,NaN,NaN,NaN,NaN
7,331de38c55626e4b86f5e0539aca962a,basketball_ncaab,NCAAB,2024-11-24T23:00:00Z,Seton Hall Pirates,Florida Atlantic Owls,2024-11-24 15:02:01,1.69,2.25,1.71,...,1.58,2.34,1.64,2.28,1.67,2.30,NaN,NaN,NaN,NaN
8,3362ad6cbedbf69a77f3147a0922bd2a,basketball_ncaab,NCAAB,2024-11-24T18:00:00Z,Fairfield Stags,Vermont Catamounts,2024-11-24 15:02:01,NaN,NaN,NaN,...,1.22,4.15,1.23,4.30,NaN,NaN,NaN,NaN,NaN,NaN
9,375204fb0f0e8be26f93e118e05f163d,basketball_ncaab,NCAAB,2024-11-24T19:30:00Z,Milwaukee Panthers,St. Thomas (MN) Tommies,2024-11-24 15:02:01,2.25,1.69,2.23,...,2.22,1.64,2.15,1.67,2.25,1.69,NaN,NaN,NaN,NaN
